# Implement Attention from Scratch
### Problem Statement
Implement a **Scaled Dot-Product Attention** mechanism from scratch using PyTorch. Your mission (should you choose to accept it) is to replicate what PyTorch's built-in `scaled_dot_product_attention` does — manually. This core component is essential in Transformer architectures and helps models focus on relevant parts of a sequence. You'll test your implementation against PyTorch's native one to ensure you nailed it.


### Requirements
1. **Define the Function**:
   - Create a function `scaled_dot_product_attention(q, k, v, mask=None)` that:
     - Computes attention scores via the dot product of query and key vectors.
     - Scales the scores using the square root of the key dimension.
     - Applies an optional mask to the scores.
     - Applies softmax to convert scores into attention weights.
     - Uses these weights to compute a weighted sum of values (V).

2. **Test Your Work**:
   - Use sample tensors for query (Q), key (K), and value (V).
   - Compare the result of your custom implementation with PyTorch's `F.scaled_dot_product_attention` using an `assert` to check numerical accuracy.


### Constraints
- ❌ Do NOT use `F.scaled_dot_product_attention` inside your custom function — that defeats the whole point.
- ✅ Your implementation must handle **batch dimensions** correctly.
- ✅ Support optional **masking** for future tokens or padding.
- ✅ Use only PyTorch ops — no cheating with external attention libs.



<details>
  <summary>💡 Hint</summary>
  Use `torch.matmul()` to compute dot products and `F.softmax()` for the final attention weights. The mask (if used) should be applied **before** the softmax using `masked_fill`.
</details>


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [3]:
import torch

torch.manual_seed(42)

batch_size = 1
seq_len = 3
dim = 3

q = torch.randn(batch_size, seq_len, dim)
k = torch.randn(batch_size, seq_len, dim)
v = torch.randn(batch_size, seq_len, dim)

In [16]:
import numpy as np

def scaled_dot_product_attention(q, k, v, mask=None):
    """
    Compute the scaled dot-product attention.

    Args:
        q: Query tensor of shape (..., seq_len_q, d_k)
        k: Key tensor of shape (..., seq_len_k, d_k)
        v: Value tensor of shape (..., seq_len_k, d_v)
        mask: Optional mask tensor of shape (..., seq_len_q, seq_len_k)

    Returns:
        output: Attention output tensor of shape (..., seq_len_q, d_v)
        attention_weights: Attention weights tensor of shape (..., seq_len_q, seq_len_k)
    """
    scores = torch.einsum('btd,bsd->bts', q, k) / np.sqrt(q.shape[-1])
    if mask:
        scores = torch.where(mask==0, -torch.inf, scores)
    weights = torch.softmax(scores, dim=-1) # bts
    output = torch.einsum('bts,bsd->btd', weights, v)
    return output, weights


# Testing on data & compare
output_custom, _ = scaled_dot_product_attention(q, k, v)
print(output_custom)
output = F.scaled_dot_product_attention(q, k, v)
print(output)

assert torch.allclose(output_custom, output, atol=1e-08, rtol=1e-05) # Check if they are close enough.

tensor([[[ 0.0377, -0.3133,  0.8707],
         [ 0.4541, -0.3508,  0.8461],
         [ 0.3709, -0.2885,  0.8044]]])
tensor([[[ 0.0377, -0.3133,  0.8707],
         [ 0.4541, -0.3508,  0.8461],
         [ 0.3709, -0.2885,  0.8044]]])
